In [49]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris

# 乱数のシードを設定
torch.manual_seed(0)

# irisデータセットの読み込み
iris = load_iris()

# 入力値と目標値を抽出
x = iris.data
t = iris.target

# Tensorに変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# 入力値と目標値をまとめる
# （[[[入力値1, 入力値2, ...], 目標値1], [[入力値1, 入力値2, ...], 目標値2], ...]）
dataset = torch.utils.data.TensorDataset(x, t)

# データセット分割
# 60%:学習用, 20%:検証用, 20%:テスト用
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# ミニバッチ学習
batch_size = 10
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# 4→4→3のニューラルネットワーク
class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    
    # 順伝播
    def forward(self, x):
        # 第1層
        h = self.fc1(x) # 線形変換
        h = F.relu(h) # 活性化関数
        # 第2層
        h = self.fc2(h) # 線形変換
        return h

# エポック数（学習回数）
max_epoch = 1

# モデルのインスタンス化とデバイスへの転送
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)

# 最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in range(max_epoch):
    for batch in train_loader:
        # ミニバッチの取得
        x, t = batch
        x, t = x.to(device), t.to(device)
        
        # 順伝播
        y = net(x)
        
        # 誤差の計算
        loss = F.cross_entropy(y, t)
        # print('loss:', loss)
        
        # 正解率の計算
        y_label = torch.argmax(y, dim=1)
        accracy = torch.mean((y_label == t).float())
        # print('accuracy:', accracy)
        
        # 勾配の初期化
        optimizer.zero_grad()
        
        # 逆伝播
        loss.backward()
        
        # パラメータの更新
        optimizer.step()

# 検証
net.eval()
with torch.no_grad():
    total_loss = 0
    total_accracy = 0
    for batch in val_loader:
        x, t = batch
        x, t = x.to(device), t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)
        total_loss += loss.item()
        y_label = torch.argmax(y, dim=1)
        total_accracy += torch.mean((y_label == t).float())
    n = len(val_loader)
    print('val_loss:', total_loss / n)
    print('val_accuracy:', total_accracy / n)

val_loss: 0.6773030956586202
val_accuracy: tensor(0.7667)
